# Antarctic Circumnavigation Expedition Cruise Track data processing

## GLONASS and Trimble GPS data

Follow the steps as described here: http://epic.awi.de/48174/

Import relevant packages

In [108]:
import pandas as pd
import csv
import MySQLdb
import datetime
import math

### STEP 1 - Extract data from database

Import data from a database table into a dataframe

In [109]:
def get_data_from_database(query, db_connection):
    
    dataframe = pd.read_sql(query, con=db_connection)

    return dataframe

**GPS data**

In [110]:
query_trimble = 'select * from ship_data_gpggagpsfix where device_id=63;'
password = input()

ace


In [111]:
db_connection = MySQLdb.connect(host = 'localhost', user = 'ace', passwd = password, db = 'ace2016', port = 3306); 

gpsdb_df = get_data_from_database(query_trimble, db_connection)
#gpsdb_df_opt = optimise_dataframe(gpsdb_df_opt)

Preview data

In [112]:
gpsdb_df.sample(5)

,id,date_time,latitude,longitude,fix_quality,number_satellites,horiz_dilution_of_position,altitude,altitude_units,geoid_height,geoid_height_units,device_id,measureland_qualifier_flags_id
6916178,13609070,2017-03-16 03:15:48.100,-44.399232,14.101791,5,7,0.128,-2.663,M,28.284,M,63,1
484650,537781,2016-12-27 14:32:21.120,-46.875541,37.868421,5,8,0.128,3996.694,M,42.907,M,63,1
6691522,13161855,2017-03-13 12:50:34.730,-54.396102,3.553809,5,7,0.128,3.727,M,21.836,M,63,1
693748,930176,2016-12-30 02:54:17.380,-46.087131,48.566578,5,7,0.128,3058.520,M,43.604,M,63,1
6633153,13045059,2017-03-12 20:36:45.050,-54.421131,3.454300,5,10,0.128,5.660,M,21.781,M,63,1


Number of data points output

In [ ]:
len(gpsdb_df)

Dates and times covered by the Trimble data set: 

In [ ]:
print("Start date:", gpsdb_df['date_time'].min())
print("End date:", gpsdb_df['date_time'].max())

Output the data into monthly files (to be able to plot them to visually screen the obvious outliers).

In [ ]:
gpsdb_df.dtypes

In [ ]:
gpsdb_df['date_time_day'] = gpsdb_df['date_time'].dt.strftime('%Y-%m-%d')

In [ ]:
gpsdb_df.sample(5)

In [ ]:
days = gpsdb_df.groupby('date_time_day')
for day in days.groups:
    path = '/home/jen/ace_trimble_gps_' + str(day) + '.csv'
    days.get_group(day).to_csv(path, index=False)

**GLONASS data**

In [ ]:
query_glonass = 'select * from ship_data_gpggagpsfix where device_id=64;'

password = input()


In [ ]:
db_connection = MySQLdb.connect(host = 'localhost', user = 'ace', passwd = password, db = 'ace2016', port = 3306); 

glonassdb_df = get_data_from_database(query_glonass, db_connection)

Preview data

In [ ]:
glonassdb_df.sample(5)

Number of data points output

In [ ]:
len(glonassdb_df)

Dates and times covered by the GLONASS data set: 

In [ ]:
print("Start date:", glonassdb_df['date_time'].min())
print("End date:", glonassdb_df['date_time'].max())

Output the data into monthly files (to be able to plot them to visually screen the obvious outliers).

In [ ]:
glonassdb_df['date_time_day'] = glonassdb_df['date_time'].dt.strftime('%Y-%m-%d')

In [ ]:
glonassdb_df.sample(5)

In [ ]:
days = glonassdb_df.groupby('date_time_day')
for day in days.groups:
    path = '/home/jen/ace_glonass_' + str(day) + '.csv'
    days.get_group(day).to_csv(path, index=False)

### STEP 2 - Visual inspection

### Trimble GPS

Data in the form of the daily csv files, were imported into QGIS mapping software in order to manually visually inspect the data points. This was done at a resolution of 1:100,000.

There were no obvious outlying points. 
Number of points flagged as outliers: 0

The following sections were identified as unusual and have been classified in the table below: 
| Start date and time (UTC) | End date and time (UTC) | Potential problem |
|---------|-----------|----------|
| 2016-12-21 10:15:46.620 | 2016-12-21 10:16:35.620 | Overlapping track |
| 2016-12-21 07:37:16.470 | 2016-12-21 07:40:02.470 | Overlapping track |
| 2016-12-22 04:43:10.010 | 2016-12-22 04:43:14.010 | Overlapping track |
| 2016-12-22 09:38:40.270 | 2016-12-22 09:38:41.270 | Strange gap |
| 2016-12-23 00:15:22.060 | 2016-12-23 00:15:33.060 | Overlapping track |
| 2016-12-23 04:57:57.310 | 2016-12-23 04:57:58.310 | Large gap |
| 2016-12-23 05:05:05.310 | 2016-12-23 05:11:43.540 | Overlapping track |
| 2016-12-23 05:30:01.560 | 2016-12-23 05:30:04.560 | Large gap |
| 2016-12-25 06:04:50.900 | 2016-12-25 06:09:52.980 | Overlapping track |
| 2016-12-29 12:57:59.390 | 2016-12-29 14:57:34.730 | Strange diversion |
| 2016-12-30 04:53:24.840 | 2016-12-30 08:51:12.670 | Strange diversion, missing data |
| 2016-12-31 00:51:37.580 | 2016-12-31 00:51:39.580 | |
| 2017-01-01 11:54:29.820 | 2017-01-01 11:54:41.820 | Overlapping track |
| 2017-01-01 22:51:51.680 | 2017-01-01 22:54:41.700 | Strange deflection |
| 2017-01-01 22:56:05.700 | 2017-01-01 22:59:57.700 | Strange deflection |
| 2017-01-04 00:34:30.360 | 2017-01-04 00:34:33.360 | Large move |
| 2017-01-13 08:26:40.850 | 2017-01-13 08:30:36.840 | Strange deflection |
| 2017-03-11 06:47:22.300 | 2017-03-11 08:21:40.970 | Strange deflection |
| 2017-03-16 17:51:36.490 | 2017-03-16 18:12:21.470 | Gap with jump |
| 2017-03-17 15:29:07.620 | 2017-03-17 15:29:10.620 | Gap with jump |
| 2017-03-18 04:07:31.290 | 2017-03-18 04:07:32.290 | Gap with jump |
| 2017-03-18 12:43:22.100 | 2017-03-18 12:43:42.100 | Overlapping track |
| 2017-03-18 13:01:04.100 | 2017-03-18 19:09:28.440 | Large gap with large time difference |
| 2017-03-24 10:13:24.720 | 2017-03-24 10:13:25.720 | Gap with jump |
| 2017-03-25 10:07:08.990 | 2017-03-25 10:07:15.990 | Gap with jump |
| 2017-03-26 22:25:49.940 | 2017-03-26 22:25:50.940 | Gap with jump |
These points have not been flagged but will be returned to later on in the processing. 

### GLONASS

Data in the form of the daily csv files, were imported into QGIS mapping software in order to manually visually inspect the data points. This was done at a resolution of 1:100,000.

There were no obvious outlying points. 
Number of points flagged as outliers: 0
    
The following sections were identified as unusual and have been classified in the table below: 
| Start date and time (UTC) | End date and time (UTC) | Potential problem |
|---------|-----------|----------|
| 2017-04-06 15:26:30 | 2017-04-06 15:26:32 | Gap with jump |
| 2017-04-06 18:22:07 | 2017-04-06 18:22:09 | Gap with jump |
| 2017-04-06 21:41:21 | 2017-04-06 21:41:45 | Strange deflection |
| 2017-04-07 15:25:06 | 2017-04-07 15:25:13 | Strange deflection |
| 2017-04-08 04:12:44 | 2017-04-08 04:13:27 | Strange deflection |
| 2017-04-08 05:01:34 | 2017-04-08 05:01:49 | Strange deflection |
| 2017-04-08 18:42:12 | 2017-04-08 18:43:04 | Strange deflection |
| 2017-04-08 18:56:02 | 2017-04-08 18:57:27 | Strange deflection |
| 2017-04-08 19:00:26 | 2017-04-08 19:01:03 | Strange deflection |
| 2017-04-08 19:05:31 | 2017-04-08 19:05:46 | Strange deflection |
| 2017-04-10 02:43:40 | 2017-04-10 02:44:25 | Strange deflection |

### STEP 3 - Motion data correction

### STEP 4 - Automated data filtering

Each data point will be compared with the one before and after to automatically filter out points that are out of the conceivable range of the ship's movement.'

The second of two consecutive points to be flagged as "likely incorrect" when any of the following cases occur: 
    - speed between two points >= 20 knots
    - acceleration between two points >= 1 ms^-2
    - direction between two points >= 5 degrees

In [ ]:
# Test
datetime = "2017-03-17 11:34:26.410"
gpsdb_df[gpsdb_df.date_time == datetime].latitude.item()

In [114]:
def get_location(datetime, position_df):
    """Create a tuple of the date_time, latitude and longitude of a location in a dataframe from a given date_time."""
    
    latitude = position_df[position_df.date_time == datetime].latitude.item()
    longitude = position_df[position_df.date_time == datetime].longitude.item()
    
    location = (datetime, latitude, longitude)
    
    return location

In [ ]:
# Test
datetime = "2017-03-17 11:34:27.410"
position_df = gpsdb_df
location = get_location(datetime, position_df)
print(location)

In [115]:
def calculate_distance(origin, destination):
    """Calculate the haversine or great-circle distance in metres between two points with latitudes and longitudes, where they are known as the origin and destination."""
    
    datetime1, lat1, lon1 = origin
    datetime2, lat2, lon2 = destination
    radius = 6371  # km
    
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) \
                                                  * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c # Distance in km
    d_m = d*1000 # Distance in metres
    
    return d_m

In [116]:
# Test
origin_datetime = "2017-03-17 11:34:26.410"
destination_datetime = "2017-03-17 11:34:27.410"
position_df = gpsdb_df

origin = get_location(origin_datetime, position_df)
destination = get_location(destination_datetime, position_df)

print("Origin:", origin)
print("Destination:", destination)

distance = calculate_distance(origin, destination)
print("Distance:", distance, "m")


Origin: ('2017-03-17 11:34:26.410', -39.3727467853333, 14.4918882686667)
Destination: ('2017-03-17 11:34:27.410', -39.3726810455, 14.4918980598333)
Distance: 7.358226506278989 m


In [ ]:
def knots_two_points(location1, location2):
    """Calculate the speed in knots between two locations which are dictionaries containing latitude, longitude and date_time."""
    
    distance = calculate_distance(location1, location2)
    
    datetime_str1, lat1, lon1 = origin
    datetime_str2, lat2, lon2 = destination
    
    datetime1 = datetime.datetime.strptime(datetime_str1,"%Y-%m-%d %H:%M:%S.%f")
    datetime2 = datetime.datetime.strptime(datetime_str2,"%Y-%m-%d %H:%M:%S.%f")
    
    seconds = abs((datetime1) - (datetime2)).total_seconds()
    
    conversion = 3600/1852 # convert 1 ms-1 to knots (nautical miles per hour; 1 nm = 1852 metres)
    speed_knots = (distance/seconds) * conversion
    
    if seconds > 0:
        return speed_knots
    else:
        return "N/A"

In [ ]:
# Test
speed = knots_two_points(origin, destination)
print("Speed: ", speed, "knots")

In [ ]:
def set_utc(date_time):
    """Set the timezone to be UTC."""
    utc = datetime.timezone(datetime.timedelta(0))
    date_time = date_time.replace(tzinfo=utc)
    return date_time

In [ ]:
positions = gpsdb_df[['date_time', 'latitude', 'longitude']]

In [ ]:
positions.sample(5)

In [ ]:
#Test
df_test = gpsdb_df.head(1000)

In [ ]:
def analyse(position_df):
    """Analyse the cruise track to ensure each point lies within a reasonable distance and direction from the previous point."""
    earliest_date_time = position_df['date_time'].min()
    latest_date_time = position_df['date_time'].max()

    current_date = earliest_date_time

    count = 0
    previous_position = get_location(earliest_date_time, position_df)
    datetime_previous, latitude_previous, longitude_previous = previous_position
    
    positions = position_df[['date_time', 'latitude', 'longitude']]
    positions.sample(5)
        
    for position in positions.itertuples():
        index, date_time, latitude, longitude = position

        current_position = position[1:]

        speed_knots = knots_two_points(previous_position, current_position)

        error_message = ""

        if speed_knots == "N/A":
            error_message = "No speed?"
        elif speed_knots >= 20:
            error_message += "**** Too fast ****"

        if error_message != "":
            print("Error: ", error_message)
            print("{}   ({:.4f}, {:.4f})    {} knots".format(current_position[0], current_position[1], current_position[2], speed_knots))

        previous_position = current_position

In [ ]:
analyse(df_test)

In [117]:
def calculate_bearing(origin, destination):
    """Calculate the direction turned between two points."""
    
    datetime1, lat1, lon1 = origin
    datetime2, lat2, lon2 = destination
    
    dlon = math.radians(lon2 - lon1)
    
    bearing = math.atan2(math.sin(dlon) * math.cos(math.radians(lat2)), 
                         math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) 
                         - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(dlon))
    
    return bearing
    

In [119]:
bearing = calculate_bearing(origin, destination)
print(math.degrees(bearing))

6.567804333108732
